# <center> Warm Up

**1)** Crie a classe Televisao com os atributos ​ligada ​(inicializado com valor False)​ ​e ​canal 
(inicializado com valor  2). 

In [21]:
class Televisao:
    def __init__(self):
        self.ligada = False
        self.canal = 2

**2)** Adicione os atributos tamanho e marca à classe Televisao. Crie dois objetos Televisao e 
atribua tamanhos e marcas diferentes. Depois, imprima o valor desses atributos de 
forma a confirmar independência dos valores de cada instância (objeto).

In [22]:
class Televisao:
    def __init__(self, tamanho, marca):
        self.tamanho = tamanho
        self.marca = marca
        self.ligada = False
        self.canal = 2

tv = Televisao(55, "samsung")
outra_tv = Televisao(42, "LG")

print(tv.tamanho, tv.marca)
print(outra_tv.tamanho, outra_tv.marca)

55 samsung
42 LG


**3)** Adicione dois novos métodos muda_canal_para_cima e muda_canal_para_baixo. 
Atualmente, a classe Televisao inicializa o canal com 2. Modifique a classe televisao de 
forma a receber o canal inicial em seu construtor.

In [26]:
class Televisao:
    def __init__(self, tamanho, marca, canal=2):
        self.tamanho = tamanho
        self.marca = marca
        self.canal = canal
        self.ligada = False
    
    def muda_canal_para_cima(self):
        self.canal += 1

    def muda_canal_para_baixo(self):
        self.canal -= 1

tv = Televisao(55, "samsung")

print("canal", tv.canal)

tv.muda_canal_para_baixo()
tv.muda_canal_para_baixo()

print("canal", tv.canal)

tv.muda_canal_para_cima()

print("canal", tv.canal)

canal 2
canal 0
canal 1


**4)** Adicione mais dois atributos canal_minimo (valor padrão 1) e ​canal_maximo​ (valor 
padrão 99) e modifique a classe Televisao de forma que, se pedirmos para mudar o 
canal para baixo, além do mínimo, ela vá para o canal máximo. Se mudarmos para 
cima, além do canal máximo, que volte ao canal mínimo.

In [28]:
class Televisao:
    def __init__(self, tamanho, marca, canal=2):
        self.tamanho = tamanho
        self.marca = marca
        self.canal_minimo = 1
        self.canal_maximo = 99
        self.canal = max(self.canal_minimo, min(canal, self.canal_maximo))
        self.ligada = False
    
    def muda_canal_para_cima(self):
        self.canal += 1
        if self.canal > self.canal_maximo:
            self.canal = self.canal_minimo

    def muda_canal_para_baixo(self):
        self.canal -= 1
        if self.canal < self.canal_minimo:
            self.canal = self.canal_maximo

tv = Televisao(55, "samsung")
print(tv.tamanho, tv.marca)

print("canal", tv.canal)

tv.muda_canal_para_baixo()
tv.muda_canal_para_baixo()

print("canal", tv.canal)

tv.muda_canal_para_cima()

print("canal", tv.canal)

canal 2
canal 99
canal 1


**5)** Modifique o construtor da classe Televisao de forma que canal_minimo e canal_maximo 
seja parametros opcionais valendo respectivamente 2 e 14.

**6)** Crie duas instancias de Televisao, especificando o valor de canal_minimo e 
canal_maximo **​por nome**.

In [36]:
class Televisao:
    def __init__(self, tamanho, marca, canal=2, canal_minimo=2, canal_maximo=14):
        self.tamanho = tamanho
        self.marca = marca
        self.canal = max(canal_minimo, min(canal, canal_maximo))
        self.canal_minimo = canal_minimo
        self.canal_maximo = canal_maximo
        self.ligada = False
    
    def muda_canal_para_cima(self):
        self.canal += 1
        if self.canal > self.canal_maximo:
            self.canal = self.canal_minimo

    def muda_canal_para_baixo(self):
        self.canal -= 1
        if self.canal < self.canal_minimo:
            self.canal = self.canal_maximo

tv = Televisao(55, "samsung", canal_minimo=10, canal_maximo=15)
print(tv.tamanho, tv.marca)

print("canal", tv.canal)

tv.muda_canal_para_baixo()
print("canal", tv.canal)

tv.muda_canal_para_cima()
print("canal", tv.canal)


outra_tv = Televisao(42, "LG", canal_minimo=1, canal_maximo=3)
print(outra_tv.tamanho, outra_tv.marca)

print("canal", outra_tv.canal)

outra_tv.muda_canal_para_baixo()
print("canal", outra_tv.canal)

outra_tv.muda_canal_para_baixo()
print("canal", outra_tv.canal)

55 samsung
canal 10
canal 15
canal 10
42 LG
canal 2
canal 1
canal 3


**7)** Crie classes para representar estados e cidades. Cada estado tem um nome, sigla e 
cidades. Cada cidade tem nome e população. Escreva um programa de testes que crie 
três estados com algumas cidades em cada um. Exiba a população de cada estado 
como a soma da população de suas cidades.

In [66]:
from typing import Iterable


class Cidade:
    def __init__(self, nome, populacao):
        self.nome = nome
        self.populacao = populacao


class Estado:
    def __init__(self, nome, sigla, cidades: Iterable[Cidade]):
        self.nome = nome
        self.sigla = sigla
        self.cidades = tuple(cidades)
    
    @property
    def populacao(self):
        return sum(cidade.populacao for cidade in self.cidades)


if __name__ == "__main__":
    floripa = Cidade("Florianópolis", 508_826)
    sao_jose = Cidade("São José", 250_181)
    sto_amaro = Cidade("Santo Amaro da Imperatriz", 23_907)

    sc = Estado("Santa Catarina", "SC", (floripa, sao_jose, sto_amaro))

    sao_paulo = Cidade("São Paulo", 12_330_000)
    campinas = Cidade("Campinas", 1_213_792)
    sorocaba = Cidade("Sorocaba", 695_328)

    sp = Estado("São Paulo", "SP", (sao_paulo, campinas, sorocaba))

    curitiba = Cidade("Curitiba", 1_963_726)
    londrina = Cidade("Londrina", 575_377)
    cascavel = Cidade("Cascavel", 332_333)

    parana = Estado("Paraná", "PR", (curitiba, londrina, cascavel))

    for estado in (sc, sp, parana):
        print()

        print("nome: ", estado.nome)
        print("sigla: ", estado.sigla)
        print("populacao: ", estado.populacao)

        print("cidades:")

        for cidade in estado.cidades:
            print("-", cidade.nome, f"({cidade.populacao} habitantes)")


nome:  Santa Catarina
sigla:  SC
populacao:  782914
cidades:
- Florianópolis (508826 habitantes)
- São José (250181 habitantes)
- Santo Amaro da Imperatriz (23907 habitantes)

nome:  São Paulo
sigla:  SP
populacao:  14239120
cidades:
- São Paulo (12330000 habitantes)
- Campinas (1213792 habitantes)
- Sorocaba (695328 habitantes)

nome:  Paraná
sigla:  PR
populacao:  2871436
cidades:
- Curitiba (1963726 habitantes)
- Londrina (575377 habitantes)
- Cascavel (332333 habitantes)


**8)** Escreva uma classe Coordenada com atributos x e y, e métodos para mostrar as 
coordenadas, calcular a distancia para outra coordenada, comparar coordenadas, 
mostrar no formato coordenada polar.

In [89]:
from math import sqrt, atan, ceil, pi

def sign(n):
    return abs(n) / n if n != 0 else 0

class Coordenada:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def get_modulo(self):
        return sqrt(self.x**2 + self.y**2)
    
    def dist(self, coord: "Coordenada"):
        delta_x = self.x - coord.x
        delta_y = self.y - coord.y
        return sqrt(delta_x**2 + delta_y**2)
    
    def to_polar(self):
        distancia = self.get_modulo()

        if self.x:
            angulo = atan(self.y / self.x)
        else:
            angulo = pi / 2 * sign(self.y)
        
        if angulo < 0:
            angulo += 2*pi * ceil(-angulo / (2*pi))
        
        return CoordenadaPolar(angulo, distancia)

    def __eq__(self, value):
        if isinstance(value, Coordenada):
            return self.x == value.x and self.y == self.y
        
        return NotImplemented
    
    def __repr__(self):
        return f"Coordenada({self.x}, {self.y})"

class CoordenadaPolar:
    def __init__(self, angulo, distancia):
        self.angulo = angulo
        self.distancia = distancia

    def __repr__(self):
        return f"CoordenadaPolar(angulo={self.angulo}, distancia={self.distancia})"

if __name__ == "__main__":
    c1 = Coordenada(10, 20)
    c2 = Coordenada(3, 4)

    print(c1, c2)
    print(c1.dist(c2))

    print(c1.to_polar(), c2.to_polar())

    print(Coordenada(0, 0).to_polar())
    print(Coordenada(-10, 50).to_polar())

Coordenada(10, 20) Coordenada(3, 4)
17.46424919657298
CoordenadaPolar(angulo=1.1071487177940904, distancia=22.360679774997898) CoordenadaPolar(angulo=0.9272952180016122, distancia=5.0)
CoordenadaPolar(angulo=0.0, distancia=0.0)
CoordenadaPolar(angulo=4.90978454023457, distancia=50.99019513592785)


**9)** Escreva classes para as seguintes formas: quadrado, retângulo e círculo.

In [103]:
from math import pi

class Retangulo:
    def __init__(self, base, altura):
        self.base = base
        self.altura = altura
    
    @property
    def perimetro(self):
        return self.base*2 + self.altura*2
    
    @property
    def area(self):
        return self.base * self.altura


class Quadrado(Retangulo):
    def __init__(self, lado):
        super().__init__(lado, lado)


class Circulo():
    def __init__(self, raio):
        self.raio = raio
    
    @property
    def perimetro(self):
        return 2 * pi * self.raio

    @property
    def area(self):
        return pi * self.raio**2
    
if __name__ == "__main__":
    ret = Retangulo(10, 20)
    print(ret.perimetro, ret.area)

    quad = Quadrado(10)
    print(quad.perimetro, quad.area)

    circ = Circulo(5)
    print(circ.perimetro, circ.area)

60 200
40 100
31.41592653589793 78.53981633974483


**10)** Escreva uma classe Fracao que armazena dois inteiros, numerador e denominador.
 
**a)** Implemente metodos para somas, subtração, multiplicação e divisão de duas 
frações

**b)** Implemente o método que imprime uma fração no formato numerador / 
denominador

**c)** Implemente um método que inverte a fração

**d)** Implemente um método que retorna a fração em valor real

**e)** Implemente um método que cria uma fração (numerador/denominador) a partir 
de um número real

In [98]:
from typing import Tuple


def mdc(a: int, b: int) -> int:
    if a == 0:
        return abs(b)

    while b != 0:
        a, b = b, a % b

    return abs(a)


def mmc(a: int, b: int) -> int:
    return a * b // mdc(a, b)


def normalize(a: float, b: float) -> Tuple[int, int]:
    while a % 1 or b % 1:
        a *= 10
        b *= 10

    m = mdc(a, b)
    return int(a / m), int(b / m)


class Fracao:
    def __init__(self, numerador, denominador=1):
        numerador, denominador = normalize(numerador, denominador)

        if denominador < 0:
            numerador *= -1
            denominador *= -1

        self.numerador = numerador
        self.denominador = denominador

    @property
    def value(self):
        return self.numerador / self.denominador

    def __mul__(self, value):
        if isinstance(value, Fracao):
            return Fracao(
                self.numerador * value.numerador, self.denominador * value.denominador
            )

        if isinstance(value, (int, float)):
            return Fracao(self.numerador * value, self.denominador)

        return NotImplemented

    def __rmul__(self, value):
        return self.__mul__(value)

    def __add__(self, other):
        if isinstance(other, Fracao):
            m = mmc(self.denominador, other.denominador)
            numerador = (
                self.numerador * m / self.denominador
                + other.numerador * m / other.denominador
            )
            return Fracao(numerador, m)

        if isinstance(other, (int, float)):
            return Fracao(self.value + other)

        return NotImplemented

    def __radd__(self, other):
        return self.__add__(other)

    def __sub__(self, other):
        return self.__add__(other.__neg__())

    def __rsub__(self, other):
        negated = self.__neg__()
        return negated.__add__(other)

    def __truediv__(self, other):
        if isinstance(other, Fracao):
            return Fracao(
                self.numerador * other.denominador, self.denominador * other.numerador
            )

        if isinstance(other, (int, float)):
            return Fracao(self.numerador, self.denominador * other)

        return NotImplemented

    def __rtruediv__(self, other):
        if isinstance(other, (int, float)):
            return Fracao(self.denominador * other, self.numerador)

        return NotImplemented

    def __neg__(self):
        return Fracao(-self.numerador, self.denominador)

    def __eq__(self, other):
        if isinstance(other, Fracao):
            return (
                self.numerador == other.numerador
                and self.denominador == other.denominador
            )

        if isinstance(other, (int, float)):
            return self.value == other

        return NotImplemented

    def __repr__(self):
        return f"Fracao({self.numerador}/{self.denominador})"


if __name__ == "__main__":
    print(Fracao(1, 2), Fracao(1/2), Fracao(0.5))

    print(Fracao(1, 4) + Fracao(5, 10))
    print(Fracao(5, 3) - Fracao(1, 4))
    print(Fracao(10, 3) * Fracao(1, 5))
    print(Fracao(5, 3) / Fracao(5, 3))

    print( (Fracao(1/4) + 0.75) == 1 )

    print(repr(Fracao(3, 2)))

    print(1 / Fracao(4, 5))

    print(Fracao(3, 4).value)
    print(Fracao(0.75))

Fracao(1/2) Fracao(1/2) Fracao(1/2)
Fracao(3/4)
Fracao(17/12)
Fracao(2/3)
Fracao(1/1)
True
Fracao(3/2)
Fracao(5/4)
0.75
Fracao(3/4)


# <center> Exercícios

## Exercício 1: Banco

In [96]:
from typing import Dict, List, Tuple


class Cliente:
    def __init__(self, nome: str, telefone: int):
        self.__nome = nome
        self.__telefone = telefone

    @property
    def nome(self):
        return self.__nome

    @property
    def telefone(self):
        return self.__telefone


class Conta:
    def __init__(self, titulares: Tuple[Cliente]):
        self.__titulares = titulares
        self.__saldo = 0

    @property
    def titulares(self):
        return self.__titulares

    @property
    def saldo(self):
        return self.__saldo

    def depositar(self, valor: float):
        self.__saldo += valor

    def sacar(self, valor: float):
        if valor > self.__saldo:
            raise ValueError("saldo insuficiente")

        self.__saldo -= valor


class ContaPoupanca(Conta):
    def __init__(self, *args, taxa=0, **kwargs):
        super().__init__(*args, **kwargs)

        self.__taxa_rendimento = taxa

    @property
    def taxa_rendimento(self):
        return self.__taxa_rendimento

    def render(self):
        self.depositar(self.saldo * self.__taxa_rendimento)


class Banco:
    def __init__(self, nome: str):
        self.__nome = nome

        self.__contas: List[Conta] = []
        self.__clientes: Dict[int, Cliente] = {}
        self.__taxa_rendimento = 0.065

    @property
    def nome(self):
        return self.__nome

    def cadastrar_cliente(self, nome: str, telefone: int):
        if nome in self.__clientes:
            raise ValueError(f"cliente '{nome}' já está cadastrado")

        cliente = Cliente(nome, telefone)
        self.__clientes[nome] = cliente

        return cliente

    def get_cliente(self, nome: str):
        return self.__clientes[nome]

    def abrir_conta(self, titulares: Tuple[Cliente], poupanca=False):
        if poupanca:
            conta = ContaPoupanca(titulares, taxa=self.__taxa_rendimento)
        else:
            conta = Conta(titulares)

        self.__contas.append(conta)

        return conta

    def get_contas(self, titular: Cliente):
        return tuple(conta for conta in self.__contas if titular in conta.titulares)

    def rendimento_mensal(self):
        for conta in self.__contas:
            if isinstance(conta, ContaPoupanca):
                conta.render()


if __name__ == "__main__":
    banco = Banco("Tatu")

    fulano = banco.cadastrar_cliente("Fulano de Tal", 48999887766)
    flavinho = banco.cadastrar_cliente("Flavinho do Pneu", 31415926535)

    try:
        banco.cadastrar_cliente("Fulano de Tal", 12312442112)
    except ValueError as e:
        print(e)

    conta = banco.abrir_conta((fulano, flavinho))

    print(conta.titulares)
    print(conta.saldo)

    try:
        conta.saldo = 1_000_000
    except AttributeError as e:
        print(e)

    conta.depositar(500)
    print(conta.saldo)

    conta.sacar(119.99)
    print(conta.saldo)

    try:
        conta.sacar(400)
    except ValueError as e:
        print(e)

    joao = banco.cadastrar_cliente("João Silva", 13987374627)
    poupanca_joao = banco.abrir_conta((joao,), poupanca=True)

    poupanca_joao.depositar(100)

    print(poupanca_joao.saldo)

    banco.rendimento_mensal()

    print(poupanca_joao.saldo)


    banco.abrir_conta((joao,))

    for conta in banco.get_contas(joao):
        print(f"{joao.nome} tem R${conta.saldo}")


cliente 'Fulano de Tal' já está cadastrado
(<__main__.Cliente object at 0x0000017B2D42BB80>, <__main__.Cliente object at 0x0000017B2D428E80>)
0
can't set attribute 'saldo'
500
380.01
saldo insuficiente
100
106.5
João Silva tem R$106.5
João Silva tem R$0


## Exercício 3: Polinômio

In [9]:
from itertools import zip_longest
from typing import Any, Iterable, Tuple


class Polinomio:
    def __init__(self, *coefs: Tuple[float]):
        if isinstance(coefs[0],  Iterable):
            coefs = coefs[0]

        self.__coefs = tuple(coefs)

    @property
    def coefs(self):
        return self.__coefs

    @property
    def grau(self):
        return len(self.__coefs) - 1

    def evaluate(self, x: float):
        return sum(coef * x**i for i, coef in enumerate(self.__coefs))
    
    def plot(self, start: float, end: float, num: int):
        step = (end - start) / num

        return tuple(self.evaluate(start + step * i) for i in range(num))

    def __getitem__(self, potencia: int):
        return self.__coefs[potencia]
    
    def __call__(self, x: float):
        return self.evaluate(x)

    def __add__(self, value: Any):
        if isinstance(value, Polinomio):
            return Polinomio(
                a + b for a, b in zip_longest(self.coefs, value.coefs, fillvalue=0)
            )
    
    def __mul__(self, value: Any):
        if isinstance(value, Polinomio):
            size = len(self.coefs) + len(value.coefs) - 1
            coefs = [0] * size

            for i, a in enumerate(self.coefs):
                for j, b in enumerate(value.coefs):
                    coefs[i + j] += a * b
            
            return Polinomio(coefs)

        if isinstance(value, (int, float)):
            return Polinomio(
                coef * value for coef in self.__coefs
            )
    
    def __rmul__(self, value: Any):
        return self.__mul__(value)
    
    def __eq__(self, value: Any):
        if isinstance(value, Polinomio):
            return self.coefs == value.coefs
    
    def __repr__(self):
        return f"Polinomio(coefs={self.coefs})"


if __name__ == "__main__":
    quad = Polinomio(10, 5, 2)  # 2x^2 + 5x + 10

    print(quad)

    print(quad.coefs)
    print(quad.grau)

    print(quad[2], quad[1], quad[0])

    print(quad.evaluate(0))
    print(quad(0))

    cubic = Polinomio(4, 2, 1, 0.5) # (1/2)x^3 + x^2 + 2x + 4
    print(cubic)

    print(cubic(0))
    print(cubic(2))

    print(quad + cubic)

    t = Polinomio(1, 1) * Polinomio(1, -1) # (1 + x)(1 - x)
    print(t)
    print(t == Polinomio(1, 0, -1)) # (1 + x)(1 - x) = 1 - x²
    
    print(cubic.plot(-10, 10, 20))

Polinomio(coefs=(10, 5, 2))
(10, 5, 2)
2
2 5 10
10
10
Polinomio(coefs=(4, 2, 1, 0.5))
4.0
16.0
Polinomio(coefs=(14, 7, 3, 0.5))
Polinomio(coefs=(1, 0, -1))
True
(-416.0, -297.5, -204.0, -132.5, -80.0, -43.5, -20.0, -6.5, 0.0, 2.5, 4.0, 7.5, 16.0, 32.5, 60.0, 101.5, 160.0, 238.5, 340.0, 467.5)


## Exercício 4: Séries

In [1]:
class Math:
    def __init__(self):
        self.__fib_cache = {}
        self.__fact_cache = {}

    def fibonacci(self, n: int):
        if n <= 1:
            return 1

        if n not in self.__fib_cache:
            self.__fib_cache[n] = self.fibonacci(n-1) + self.fibonacci(n-2)
            
        return self.__fib_cache[n]
    
    def fatorial(self, n: int):
        if n <= 1:
            return 1
        
        if n not in self.__fact_cache:
            self.__fact_cache[n] = n * self.fatorial(n-1)
        
        return self.__fact_cache[n]

    def primos(self, n: int = float("inf")):
        """Gera os n primeiros números primos."""

        seq = [2]
        i = 3

        yield 2

        while True:
            if not any(i % p == 0 for p in seq):
                seq.append(i)
                yield i

                if len(seq) >= n:
                    break

            i += 2


if __name__ == "__main__":
    math = Math()

    print(math.fibonacci(20))

    print([math.fibonacci(i) for i in range(20)])

    print([math.fatorial(i) for i in range(10)])

    print(*math.primos(40), sep=" ")

10946
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765]
[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]
2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 101 103 107 109 113 127 131 137 139 149 151 157 163 167 173


## Exercício 9: Campo Minado

In [6]:

from typing import Tuple

class Mapa:
    def __init__(self, m: int, n: int):
        # matriz representando quais tiles já foram
        # revelados
        self.__tiles = [[False] * n for _ in range(m) ]

        # matriz representando o nível de risco de
        # cada tile (inverso da proximidade de uma bomba)
        self.__risco = [[0] * n for _ in range(m) ]

        # dicionários que mapeiam uma posição à uma
        # bomba e bandeira
        self.__bombas = {}
        self.__bandeiras = {}

        # se o jogador revelar uma posição contendo uma bomba
        self.__explodiu = False

    @property
    def tiles(self):
        return self.__tiles

    @property
    def risco(self):
        return self.__risco

    @property
    def bombas(self):
        return self.__bombas
    
    @property
    def bandeiras(self):
        return self.__bandeiras
    
    @property
    def explodiu(self):
        return self.__explodiu

    def gerar_bombas(self):
        """
        Gera a posição de todas as bombas e atualiza
        o mapa de risco.
        """
        ...
    
    def revelar(self, x: int, y: int):
        """
        Revela a posição (x, y). Recursivamente revela todos
        as posições adjacentes com nível 0 de risco.
        """
        ...
    
    def colocar_bandeira(self, x: int, y: int):
        """Coloca uma bandeira na posição (x, y)."""
        ...
    
    def retirar_bandeira(self, x: int, y: int):
        """Retira uma bandeira da posição (x, y)."""
        ...
    
    def tem_bandeira(self, x: int, y: int) -> bool:
        """Testa se na posição (x, y) há uma bandeira."""
        ...

    def tem_bomba(self, x: int, y: int) -> bool:
        """Testa se na posição (x, y) há uma bomba."""
        ...

class Menu:
    def __init__(self):
        ...
    
    def pedir_pos(self) -> Tuple[int, int]:
        """Pede ao usuário para inserir uma posição no terminal."""
        ...
    
    def imprimir_mapa(self, mapa: Mapa):
        """Imprime o mapa no terminal."""
        ...

class Jogo:
    def __init__(self):
        self.mapa_tamanho = (20, 20)

        self.mapa = Mapa(*self.mapa_tamanho)
        self.menu = Menu()
    
    def iniciar(self):
        ...

    def loop(self):
        ...

if __name__ == "__main__":
    jogo = Jogo()
    jogo.iniciar()

hi


## Exercicio 10: Organização de Turmas

In [ ]:
class Pessoa:
    def __init__(self, id: int, nome: str):
        self.__id = id
        self.__nome = nome

    @property
    def id(self):
        return self.__id

    @property
    def nome(self):
        return self.__nome


class Aluno(Pessoa):
    def __init__(self, id: int, nome: str, email: str):
        super().__init__(id, nome)
        self.__email = email
        self.__iaa = 0
    
    @property
    def iaa(self):
        return self.__iaa
    
    @property
    def email(self):
        return self.__email


class Professor(Pessoa):
    def __init__(self, id: int, nome: str):
        super().__init__(id, nome)


class Turma:
    def __init__(self, disciplina: str):
        self.__disciplina = disciplina
        self.__profs = set()
        self.__alunos = set()
    
    @property
    def disciplina(self): return self.__disciplina

    @property
    def professores(self): return self.__profs

    @property
    def alunos(self): return self.__alunos
    
    def matricular_aluno(self, aluno: Aluno):
        self.__alunos.add(aluno)

    def matricular_professor(self, professor: Professor):
        self.__profs.add(professor)


class Gerenciador:
    def __init__(self):
        self.__alunos = {}
        self.__professores = {}
        self.__turmas = {}

        self.__id = 0
    
    def gerar_id(self):
        self.__id += 1
        return self.__id        
    
    def cadastrar_aluno(self, nome: str, email: str):
        matricula = self.gerar_id()
        aluno = Aluno(matricula, nome, email)

        self.__alunos[matricula] = aluno

        return aluno

    def cadastrar_professor(self, nome: str):
        matricula = self.gerar_id()
        professor = Professor(matricula, nome)

        self.__professores[matricula] = professor

        return professor
    
    def cadastrar_turma(self, disciplina: str):
        turma = Turma(disciplina)
        self.__turmas[disciplina] = turma
        return turma


if __name__ == "__main__":

    gerenciador = Gerenciador()

    turma_calculo = gerenciador.cadastrar_turma("mtm3120")
    print(turma_calculo)

    fulano = gerenciador.cadastrar_aluno("Fulano", "fulano123@gmail.com")
    print(fulano)